In [2]:
import numpy as np
import cv2
import math
from PIL import Image, ImageStat
import sys
from os import listdir

In [14]:
"""
This is the function used to resizing the image into a smaller size. Since from the experiments, the larger size couldn't get a better 
training result.
"""
def convert(dir):
    file_list=listdir(dir)
    for filename in file_list:
        if "panorama" in filename and "cropped" not in filename and "Resized_" not in filename:
            path=""
            path=dir+filename
            im=Image.open(path)
            out=im.resize((416,208),Image.ANTIALIAS)
            print ("%s has been resized"%filename)
            out.save(dir+"Resized_"+filename)

**Put this file into the directory of each classes. The output is a '.npz' file which contains the image data.**

In [15]:
path1 = r"C:\\Users\\35252\\Desktop\\"
files= listdir(path1)

convert(path1)

panorama-1O2_CdMZC8UpnsvAnuNZTw-1.png has been resized
panorama-2PRkuSeJg_Am_3fdkpMk1w-1.png has been resized
panorama-2xbH0MV44SbMGf2X07Gpdw-1.png has been resized


**return pixels matrix with RGBA value, and cropped half image**

In [16]:
def get_image(image_path):
    """
    Get a numpy 2D array of an image so that one can access RGBA[x][y] or RGBA[x,y].
    
    """
    origin = Image.open(image_path, 'r')
    width, height = origin.size
    area = (0, 0, width, 0.5*height)
    image = origin.crop(area) # crop top half of the image
#     result = Image.fromarray(image.astype(np.uint8))
    image.save(file+"_cropped.png")
    width, height = image.size
#     print (image.size)
    pixel_values = list(image.getdata())
    if image.mode == 'RGBA':
        channels = 4
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_values = np.array(pixel_values).reshape((height, width, channels))
    return pixel_values 

**Get a numpy 2D array of an image which contains rgb values**

In [17]:
def RGB_array(pixels):
    rgb=[[0 for x in range(len(pixels[0]))] for y in range(len(pixels))]
    r,g,b=[],[],[]
    # Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
            R,G,B,A=pixels[i,j]
            rgb[i][j]=[R,G,B]
            r.append(R)
            g.append(G)
            b.append(B)
    rgb2 = r+g+b
    return rgb2

In [18]:
train_x = []
for file in sorted(files):
    if "panorama" in file and "Resized" in file and "cropped" not in file:
#         file=path1+file
        pixels =get_image(file)
#         print (np.shape(pixels))
            # print (pixel1)
        image_rgb = RGB_array(pixels)
        
#             edge_cal(file)
        image_rgb=np.asarray(image_rgb)
        print(np.shape(image_rgb))
#         image_rgb=image_rgb.reshape((43264,3))
        train_x.append(image_rgb)
        print(file,np.shape(train_x))

(129792,)
Resized__resized__resized__resized__resized_panorama-2PRkuSeJg_Am_3fdkpMk1w-1.png (1, 129792)
(129792,)
Resized__resized__resized__resized__resized_panorama-2xbH0MV44SbMGf2X07Gpdw-1.png (2, 129792)
(129792,)
Resized_panorama-1O2_CdMZC8UpnsvAnuNZTw-1.png (3, 129792)
(129792,)
Resized_panorama-2PRkuSeJg_Am_3fdkpMk1w-1.png (4, 129792)
(129792,)
Resized_panorama-2xbH0MV44SbMGf2X07Gpdw-1.png (5, 129792)


**Here the number of slice could be change which depends on your size of data**

In [13]:
train_X = train_x[0:100]
test_X = train_x[100:120]
print(np.shape(train_X))
print(np.shape(test_X))

(100, 129792)
(20, 129792)


In [14]:
train_Y,test_Y = [],[]
for i in range(100):
    train_Y.append(1)
for i in range(20):
    test_Y.append(1)

**The output contains 4 matrixs. In each metrixs, each row represents an immage. The value in eahc row represents the rgb values**

In [15]:
np.savez_compressed("BigBuilding", train_x = train_X, train_Y = train_Y,test_x = test_X, test_y = test_Y);